In [1]:
import numpy as np
from scipy.linalg import expm
from matplotlib import pyplot as plt
import cirq
from cirq import H, CNOT, X

# Setting up the $D_4$ group multiplication structure

The first cell defines all the correct multiplication matrices and handles the conjugacy classes, etc. The eight elements are encoded in 8-dimensional unit vectors as follows:

e      [1,0,0,0,0,0,0,0]

R      [0,1,0,0,0,0,0,0]

R2     [0,0,1,0,0,0,0,0]

R3     [0,0,0,1,0,0,0,0]

m      [0,0,0,0,1,0,0,0]

mR     [0,0,0,0,0,1,0,0]

mR2    [0,0,0,0,0,0,1,0]

mR3    [0,0,0,0,0,0,0,1]

This correctly corresponds to the kronecker products of the qubit encoding in the computational basis

In [2]:
# These permutation matrices compute left multiplication and right multiplication on the group elements as column vectors


e = np.identity(8)

R_l = np.array([[0,0,0,1,0,0,0,0],
                [1,0,0,0,0,0,0,0],
                [0,1,0,0,0,0,0,0],
                [0,0,1,0,0,0,0,0],
                [0,0,0,0,0,1,0,0],
                [0,0,0,0,0,0,1,0],
                [0,0,0,0,0,0,0,1],
                [0,0,0,0,1,0,0,0]])

R2  = np.array([[0,0,1,0,0,0,0,0],
                [0,0,0,1,0,0,0,0],
                [1,0,0,0,0,0,0,0],
                [0,1,0,0,0,0,0,0],
                [0,0,0,0,0,0,1,0],
                [0,0,0,0,0,0,0,1],
                [0,0,0,0,1,0,0,0],
                [0,0,0,0,0,1,0,0]])

R3_l = np.array([[0,1,0,0,0,0,0,0],
                 [0,0,1,0,0,0,0,0],
                 [0,0,0,1,0,0,0,0],
                 [1,0,0,0,0,0,0,0],
                 [0,0,0,0,0,0,0,1],
                 [0,0,0,0,1,0,0,0],
                 [0,0,0,0,0,1,0,0],
                 [0,0,0,0,0,0,1,0]])

m_l = np.array([[0,0,0,0,1,0,0,0],
                [0,0,0,0,0,1,0,0],
                [0,0,0,0,0,0,1,0],
                [0,0,0,0,0,0,0,1],
                [1,0,0,0,0,0,0,0],
                [0,1,0,0,0,0,0,0],
                [0,0,1,0,0,0,0,0],
                [0,0,0,1,0,0,0,0]])

mR_l = np.array([[0,0,0,0,0,1,0,0],
                 [0,0,0,0,0,0,1,0],
                 [0,0,0,0,0,0,0,1],
                 [0,0,0,0,1,0,0,0],
                 [0,0,0,1,0,0,0,0],
                 [1,0,0,0,0,0,0,0],
                 [0,1,0,0,0,0,0,0],
                 [0,0,1,0,0,0,0,0]])

mR2_l = np.array([[0,0,0,0,0,0,1,0],
                  [0,0,0,0,0,0,0,1],
                  [0,0,0,0,1,0,0,0],
                  [0,0,0,0,0,1,0,0],
                  [0,0,1,0,0,0,0,0],
                  [0,0,0,1,0,0,0,0],
                  [1,0,0,0,0,0,0,0],
                  [0,1,0,0,0,0,0,0]])

mR3_l = np.array([[0,0,0,0,0,0,0,1],
                  [0,0,0,0,1,0,0,0],
                  [0,0,0,0,0,1,0,0],
                  [0,0,0,0,0,0,1,0],
                  [0,1,0,0,0,0,0,0],
                  [0,0,1,0,0,0,0,0],
                  [0,0,0,1,0,0,0,0],
                  [1,0,0,0,0,0,0,0]])

R_r = np.array([[0,0,0,1,0,0,0,0],
                [1,0,0,0,0,0,0,0],
                [0,1,0,0,0,0,0,0],
                [0,0,1,0,0,0,0,0],
                [0,0,0,0,0,0,0,1],
                [0,0,0,0,1,0,0,0],
                [0,0,0,0,0,1,0,0],
                [0,0,0,0,0,0,1,0]])

R3_r = np.array([[0,1,0,0,0,0,0,0],
                 [0,0,1,0,0,0,0,0],
                 [0,0,0,1,0,0,0,0],
                 [1,0,0,0,0,0,0,0],
                 [0,0,0,0,0,1,0,0],
                 [0,0,0,0,0,0,1,0],
                 [0,0,0,0,0,0,0,1],
                 [0,0,0,0,1,0,0,0]])

m_r = np.array([[0,0,0,0,1,0,0,0],
                [0,0,0,0,0,0,0,1],
                [0,0,0,0,0,0,1,0],
                [0,0,0,0,0,1,0,0],
                [1,0,0,0,0,0,0,0],
                [0,0,0,1,0,0,0,0],
                [0,0,1,0,0,0,0,0],
                [0,1,0,0,0,0,0,0]])

mR_r = np.array([[0,0,0,0,0,1,0,0],
                 [0,0,0,0,1,0,0,0],
                 [0,0,0,0,0,0,0,1],
                 [0,0,0,0,0,0,1,0],
                 [0,1,0,0,0,0,0,0],
                 [1,0,0,0,0,0,0,0],
                 [0,0,0,1,0,0,0,0],
                 [0,0,1,0,0,0,0,0]])

mR2_r = np.array([[0,0,0,0,0,0,1,0],
                  [0,0,0,0,0,1,0,0],
                  [0,0,0,0,1,0,0,0],
                  [0,0,0,0,0,0,0,1],
                  [0,0,1,0,0,0,0,0],
                  [0,1,0,0,0,0,0,0],
                  [1,0,0,0,0,0,0,0],
                  [0,0,0,1,0,0,0,0]])

mR3_r = np.array([[0,0,0,0,0,0,0,1],
                  [0,0,0,0,0,0,1,0],
                  [0,0,0,0,0,1,0,0],
                  [0,0,0,0,1,0,0,0],
                  [0,0,0,1,0,0,0,0],
                  [0,0,1,0,0,0,0,0],
                  [0,1,0,0,0,0,0,0],
                  [1,0,0,0,0,0,0,0]])

# This function transforms group elements from string format (as written at start of file) to column vector format used in the code. 
# This makes some later functions more readable.

def vector(element):
    if element in ['e']:
        return np.array([1,0,0,0,0,0,0,0])
    elif element in ['R']:
        return np.array([0,1,0,0,0,0,0,0])
    elif element in ['R2']:
        return np.array([0,0,1,0,0,0,0,0])
    elif element in ['R3']:
        return np.array([0,0,0,1,0,0,0,0])
    elif element in ['m']:
        return np.array([0,0,0,0,1,0,0,0])
    elif element in ['mR']:
        return np.array([0,0,0,0,0,1,0,0])
    elif element in ['mR2']:
        return np.array([0,0,0,0,0,0,1,0])
    elif element in ['mR3']:
        return np.array([0,0,0,0,0,0,0,1])

def conjugacy_class(element):
    if element in ['e']:
        return [vector('e')]
    elif element in ['R2']:
        return [vector('R2')]
    elif element in ['R','R3']:
        return [vector('R'),vector('R3')]
    elif element in ['m','mR2']:
        return [vector('m'),vector('mR2')]
    elif element in ['mR','mR3']:
        return [vector('mR'),vector('mR3')]

# Gets the correct matrices for left and right multiplication

def left_mult(element):
    if np.array_equal(element,np.array([1,0,0,0,0,0,0,0])) == True:
        return e
    elif np.array_equal(element,np.array([0,1,0,0,0,0,0,0])) == True:
        return R_l
    elif np.array_equal(element,np.array([0,0,1,0,0,0,0,0])) == True:
        return R2
    elif np.array_equal(element,np.array([0,0,0,1,0,0,0,0])) == True:
        return R3_l
    elif np.array_equal(element,np.array([0,0,0,0,1,0,0,0])) == True:
        return m_l
    elif np.array_equal(element,np.array([0,0,0,0,0,1,0,0])) == True:
        return mR_l
    elif np.array_equal(element,np.array([0,0,0,0,0,0,1,0])) == True:
        return mR2_l
    elif np.array_equal(element,np.array([0,0,0,0,0,0,0,1])) == True:
        return mR3_l

def right_mult(element):
    if np.array_equal(element,np.array([1,0,0,0,0,0,0,0])) == True:
        return e
    elif np.array_equal(element,np.array([0,1,0,0,0,0,0,0])) == True:
        return R_r
    elif np.array_equal(element,np.array([0,0,1,0,0,0,0,0])) == True:
        return R2
    elif np.array_equal(element,np.array([0,0,0,1,0,0,0,0])) == True:
        return R3_r
    elif np.array_equal(element,np.array([0,0,0,0,1,0,0,0])) == True:
        return m_r
    elif np.array_equal(element,np.array([0,0,0,0,0,1,0,0])) == True:
        return mR_r
    elif np.array_equal(element,np.array([0,0,0,0,0,0,1,0])) == True:
        return mR2_r
    elif np.array_equal(element,np.array([0,0,0,0,0,0,0,1])) == True:
        return mR3_r


Below a set of functions used in the classical shadows protocol. The first sample angles with the correct distributions to then assemble random unitary matrices. The next cell then attaches the classical shadows operations (i.e. random unitaries and measurements) to a circuit of interest, with a given size. It is easiest to make circuits with ancillas at the end, these will then not be measured. Finally, it includes the function which uses the unitaries to reconstruct the 'snapshot' of the system from the measurements.

In [3]:
ident = np.array([[1., 0.], [0., 1.]])
sigma_x = np.array([[0., 1.], [1., 0.]])
sigma_y = np.array([[0., -1.0j], [1.0j, 0.]])
sigma_z = np.array([[1., 0.], [0., -1.]])

In [4]:
# Random angles correctly sampled for the Haar measure

def omega():
    return np.random.rand() * np.pi * 2
def phi ():
    return np.random.rand() * np.pi * 2
def theta():
    return np.arccos(2 * np.random.rand() - 1)

def unitary_angles(n_qubits):
    omegas = [omega() for i in range(n_qubits)]
    phis = [phi() for i in range(n_qubits)]
    thetas = [theta() for i in range(n_qubits)]

    return [phis,omegas,thetas]

# Unitary matrix constructed from three angles

def unitary(phi,omega,theta):
    
    return expm(-0.5j * omega * sigma_z) @ expm(-0.5j * theta * sigma_x) @ expm(-0.5j * phi * sigma_z)

# Projectors onto |0> and |1> states for an invidual qubit
 
projectors = np.zeros([2,2,2])
projectors[0,0,0] = 1
projectors[1,1,1] = 1

In [5]:
# Given an n-qubit circuit, attach the relevant random unitaries and measurements

def shadows(qubits,circuit,angles,n_qubits):          # applies unitaries and measurement to first n qubits (i.e. put ancillas at the end)

    shadows = cirq.Circuit()

    # Attach circuit of interest

    shadows.append(circuit)


    # Random unitaries nitaries

    shadows.append([cirq.rz(angles[0][i])(qubits[i]) for i in range(n_qubits)],strategy=cirq.circuits.InsertStrategy.NEW_THEN_INLINE)
    shadows.append([cirq.rx(angles[2][i])(qubits[i]) for i in range(n_qubits)],strategy=cirq.circuits.InsertStrategy.NEW_THEN_INLINE)
    shadows.append([cirq.rz(angles[1][i])(qubits[i]) for i in range(n_qubits)],strategy=cirq.circuits.InsertStrategy.NEW_THEN_INLINE)

    # Measuring

    shadows.append(cirq.measure(*qubits[0:n_qubits],key='out'))

    return shadows

# Reconstruct the 'snapshot' of the density matrix using unitary matrices

def snapshot(angles,results):
    
    rho = np.eye(1)

    for i_mmt, mmt in enumerate(results):
        U = unitary(angles[0][i_mmt],angles[1][i_mmt],angles[2][i_mmt])
        
        rho = np.kron(rho, 3 * np.conj((U.T)) @ projectors[:,:,mmt] @ U - np.eye(2))
    
    return rho

# Example: GHZ circuit

GHZ_qubits = cirq.LineQubit.range(9)

GHZ_circuit = cirq.Circuit(cirq.H(GHZ_qubits[0]),
                           [CNOT(GHZ_qubits[i], GHZ_qubits[i+1]) for i in range(8)])

# expected density matrix for 9 qubit GHZ
rho_ex = np.zeros([512,512])
rho_ex[0,0] = 1/2
rho_ex[511,511] = 1/2
rho_ex[0,511] = 1/2
rho_ex[511,0] = 1/2


Below cell sets up the circuit of interest, on the three-edge lattice, and calculates the expected resultant density matrix.

In [6]:
# More interesting example, the three edge setup with an ancilla and two plaquette defects

fusion_qubits = cirq.LineQubit.range(11)
g1_m, g1_R2, g1_R, g2_m, g2_R2, g2_R, g3_m, g3_R2, g3_R, x1, x2 = fusion_qubits

fusion = cirq.Circuit()

# Prepare ground state
fusion.append([
    H(g1_m),
    H(g1_R2),
    H(g2_R)
])

fusion.append([
    CNOT(g1_m,g2_m),
    CNOT(g1_m,g3_m),
    CNOT(g1_R2,g2_R2),
    CNOT(g1_R2,g3_R2),
    CNOT(g1_R,g2_R),
    CNOT(g1_R,g3_R),
])

# Prepare the ancilla in the superposition of the conjugacy class and multiply onto g1 and g2

fusion.append([
    H(x1),
    H(x2)
])

fusion.append([
    CNOT(x1,g1_R2),
    X(g1_m),

    CNOT(x2,g2_R2),
    X(g2_m)
])

# expected density matrix for the resulting state
state = np.zeros(2**9)

for i in range(8):
    g = np.zeros(8)
    g[i] = 1

    for x1 in conjugacy_class('m'):
        for x2 in conjugacy_class('m'):

            state += np.kron(right_mult(x1) @ g, np.kron(right_mult(x2) @ g, g))

state /= np.sqrt(32)

rho_ex2 = np.outer(state,state)

Finally, the cell which uses all the previous functions to approximate the density matrix as an average over many snapshots.

In [7]:
n_qubits = 9

qubits = fusion_qubits
circuit = fusion

N_rep = 100

for i in range(10):
    rho = np.zeros([2**n_qubits,2**n_qubits],dtype='complex')

    for repetitions in range(N_rep):
        
        angles = unitary_angles(n_qubits)
        
        shadow_circuit = shadows(qubits,circuit,angles,n_qubits)

        simulator = cirq.Simulator()
        result = simulator.run(shadow_circuit).measurements['out'][0]

        rho += snapshot(angles,result)

    rho /= N_rep


    print(np.trace(np.conj((rho-rho_ex2).T)@(rho - rho_ex2)))

(1964.482613237615+0j)


In [69]:
n_qubits = 9

qubits = fusion_qubits
circuit = fusion

N_rep = 1000

for i in range(10):
    rho = np.zeros([2**n_qubits,2**n_qubits],dtype='complex')

    for repetitions in range(N_rep):
        
        angles = unitary_angles(n_qubits)
        
        shadow_circuit = shadows(qubits,circuit,angles,n_qubits)

        simulator = cirq.Simulator()
        result = simulator.run(shadow_circuit).measurements['out'][0]

        rho += snapshot(angles,result)

    rho /= N_rep


    print(np.trace(np.conj((rho-rho_ex2).T)@(rho - rho_ex2)))



(1953.2395310028974+0j)
(1952.741325194952+0j)
(1949.0419459528975+0j)
(1955.8739572086301+0j)
(1954.481696135281+0j)
(1967.5062938487015+0j)
(1930.6966436476987+0j)
(1967.270254174688+0j)
(1957.163183149365+0j)
(1954.5461403624195+0j)


In [67]:
def the_outer_loop(input):
    angles = unitary_angles(n_qubits)
    shadow_circuit = shadows(qubits,circuit,angles,n_qubits)
    simulator = cirq.Simulator()
    result = simulator.run(shadow_circuit).measurements['out'][0]
    return snapshot(angles,result) 

In [68]:
from joblib import Parallel, delayed
N_rep = 10
rho_snaps = Parallel(n_jobs=16)(delayed(the_outer_loop)(x) for x in range(N_rep))
rho = np.array(rho_snaps).mean(axis=0)
print(np.trace(np.conj((rho-rho_ex2).T)@(rho - rho_ex2)))

(194851.57409028144+0j)
